In [1]:
# Lo primero sera importar las librerias y estableces algunas configuraciones para
# facilitar la lectura en pantalla

import pandas as pd
import numpy as np
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)

In [69]:
# leer los datos y cargarlos en dataframe, validando la carga al presentar informacion basica
# del archivo
mensajes = pd.read_json('C:/Users/Luis/Documents/Ciencia de Datos/Dataset1/mensajespd.json')

print(mensajes.shape)

# borramos registros duplicados desde los datos originales
mensajes.drop_duplicates(inplace=True)

print(mensajes.head())
print("\n")
print("Cantidad de registros de mensajes = %3d" % mensajes.shape[0])
print("\n")
print(mensajes.columns)

(1168, 3)
   Enviado/por           Fecha/Hora  \
0  04140819214  29/08/2018 16:31:19   
1  04140819214  30/08/2018 08:34:25   
2  04145328570  28/08/2018 18:03:12   
3  04145328570  28/08/2018 18:03:19   
4  04145328570  01/09/2018 10:15:48   

                                                                                                                                                          Texto  
0  Luis que si le mando la plata de bicentimario 600 bolibares soberanos ala cuenta mercantil                                                                    
1  ps luis qe si mamdo la plata que esta en mi cuenta vecentenario a la mercantil                                                                                
2  Hola Dios te bendiga                                                                                                                                          
3  No hay internet                                                                                         

In [70]:
# ahora se va a proceder a colocar en minuscula, todo el contenido de la variable "Texto"
# todo a minuscula

mensajes.Texto = mensajes.Texto.str.lower()

# y luego eliminamos todos los acentos 

import re
import unicodedata

def strip_accents(text):
    """
    Strip accents from input String.

    :param text: The input string.
    :type text: String.

    :returns: The processed String.
    :rtype: String.
    """
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError): # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

mensajes["Texto"] = mensajes["Texto"].apply(lambda tx: strip_accents(tx))



**el dataset requiere depuracion ya que multiples
 registros (algunos), contienen informacion de un solo mensaje, lo que distorciona la
 interpretacion de dichos mensajes
 primer paso ordenar el dataframe por "Enviado/por" y "Fecha/Hora"**

In [71]:
# ordenamos
mensajes.sort_values(by=["Enviado/por", "Fecha/Hora"])

# reacomodando indices
indice = list(range(mensajes.shape[0]))
mensajes.index = indice

# convertimos a formato de fecha la columna "Fecha/Hora"
mensajes["Fecha/Hora"] = pd.to_datetime(mensajes["Fecha/Hora"])

# variable de interes tiempo en segundos (hora del dia en segundos)

mensajes['ts'] = mensajes["Fecha/Hora"].apply(lambda x: x.hour*60*60+x.minute*60+x.second)

# el tiempo entre mensajes: tem
# voala! funciona.. 

mensajes["tem"] = np.zeros(mensajes.shape[0], dtype=int)

mensajes["tem"] = abs(mensajes["ts"].diff())



In [72]:
# con este arreglo de indice calculado a continuacion, concatenamos 
# mensajes divididos en la fila i-1 y posterior eliminamos los (i) ndice
# ya q seran redundantes
# hay q tener en cuenta filtrar q mensajes a concatenar sean del mismo
# sender y que contenido de mensajes sean distintos

# cosideramos solo aquellos mensajes sucesivos que hayan sido recibidos
# en un intervalo de tiempo menor a 10 segundos ya que en este subgrupo se encuentran
# los mensajes divididos

import warnings
warnings.filterwarnings('ignore')

indimulmensajes = mensajes[mensajes["tem"]<10].index

# uniendo y borrando registros redundantes, despues de unir mensajes fragmentados.
# Hay q recorrer el dataset de abajo hacia arriba (reverse) dado que: puede haber 
# mensajes fraccionados en mas de 2 partes y a la vez es posible borrar cada
# registro redundante una vez que se unan.

import operator
for i in reversed(indimulmensajes):
    if operator.and_(mensajes["Enviado/por"][i-1]==mensajes["Enviado/por"][i], mensajes["Texto"][i-1]!=mensajes["Texto"][i]):
        mensajes["Texto"][i-1] += mensajes["Texto"][i]
        mensajes.drop(mensajes.index[i], inplace = True)


# reacomodando indices
indice = list(range(mensajes.shape[0]))
mensajes.index = indice
        
       
# guardamos todo el nuevo dataframe con datos limpios, preprocesados
mensajes.to_pickle("C:/Users/Luis/Documents/Ciencia de Datos/Dataset1/mensajes.pkl")

In [75]:
# cargamos el dataframe mensajes con datos limpios, preprocesados

mensajes = pd.read_pickle("C:/Users/Luis/Documents/Ciencia de Datos/Dataset1/mensajes.pkl")


print(mensajes.Texto[mensajes.ts==24468])




27    buenos dias sr luis feliz inicio de semana como esta, que pena molestarlo tan temprano pero desde el sabado en la noche estoy sin internet, dice sin accso a internet por favor es que usted sabe que yo trabajo desde la casa y ayer no pude hacer nada soy omaira perni
Name: Texto, dtype: object


**es_suscriptor: esta variable toma valor 0 si el contenido de "Texto" proviene de un
cliente de la red comunitaria, y 1 en caso contrario. Pero ¿Como se determina si el 
mensaje proviene o no de un suscriptor?**
**a vuelo de pajaro podemos decir que:**

**1- Si hacemos query de "Enviado/por" en la base de datos de cliente y aparece en esta
definitivamente el mensaje fue enviado por un suscriptor, es el caso mas sencillo**

**2- Que pasa si el query no devuelve datos, quiere decir que no se encuentra registrado
pero esto no quiere decir que el mensaje no provenga de un cliente, sino que tal vez
este usando otro dispositivo no registrado en la base de datos, de alli que surga la necesidad
de analizar el contenido para establecer el valor de la variable es_suscriptor.
en este apartado se puede inferir que tal vez el usuario se identifique con su nombre
dentro del contenido del mensaje o tal vez no, en caso de que si, la solucion pasaria algo
similar a 1, en caso contrario se necesitara establecer mas precision en cuanto al contenido
de "Texto" y hasta sera necesario interpretar varios mensajes simultaneamente del mismo 
Enviado/por" en un tiempo-estipulado e interactuar con el remintente para clasificarlo.**

**3- Otro aspecto que pareciera ser prometedor es la frecuencia de origen de sms, obtenida de 
la variable "Enviado/por"**


In [8]:
# creando 2 variables, una que indique si "Texto" proviene de un suscriptor del servicio
# de internet (0) o no (1) y otra que partiendo del contenido de "Texto" clasifique en mensaje,
# lo cual se hara luego de analizar la variable "Texto" mas adelante

# por defecto ninguno es suscriptor
mensajes["es_suscriptor"] = np.ones(mensajes.shape[0], dtype=int)

# aun no se a establecido criterio para clasificar, se inicializa en 0
mensajes["tipo_mensaje"] = np.zeros(mensajes.shape[0], dtype=int)

In [9]:
# leer telefonos conocidos de clientes desde archivo
# al archivo tlfclientes.csv se le agrego un caracter alfanumerico al comienzo
# de cada registro para que pandas leyera como texto y no como numero

telef_clientes = pd.read_csv('C:/Users/Luis/Documents/Ciencia de Datos/Dataset1/tlfclientes.csv')
print(telef_clientes.shape)
print(telef_clientes.head())
print(telef_clientes.columns)

# jugando un poco con los dataframe, obteniendo salidas de datos relacionadas
#print(mensajes["Enviado/por"][4])

#print(telef_clientes.telef[37][1:-1])

#print(telef_clientes["telef"][1:-1])

#print(telef_clientes[telef_clientes.telef==mensajes["Enviado/por"][4]])

#print(mensajes.Texto[mensajes["Enviado/por"]==telef_clientes.telef[37][1:-1]])


# ahora se convierte telef_clientes.telef eliminando los caracteres- al inicio y final
telef_clientes["telef"] = telef_clientes["telef"].str.replace("-","")

# y se carga a memoria como lista, ya q son pocos
lista_telef = telef_clientes["telef"].tolist()

print(len(lista_telef))

                                          

(120, 1)
           telef
0  -04147062906-
1  -04247807722-
2  -04247657429-
3  -04120713877-
4  -04266670618-
Index(['telef'], dtype='object')
120


In [10]:
# ahora clasificamos como 0 la variable "es_suscriptor", si coincide lista con 
# "Enviado/por" del dataframe

# print(mensajes.Texto[mensajes["Enviado/por"].isin(lista_telef)])

mensajes.loc[mensajes["Enviado/por"].isin(lista_telef), "es_suscriptor"] = 0 

# validamos con un crosstab, bien ya tenemos parte de los mensajes clasificados

tab = pd.crosstab(index=mensajes["es_suscriptor"],columns="frecuencia")
print(tab)
print("\n", "conteo")
#print(mensajes["es_suscriptor"].value_counts())
print(mensajes.es_suscriptor.value_counts())



col_0          frecuencia
es_suscriptor            
0              259       
1              789       

 conteo
1    789
0    259
Name: es_suscriptor, dtype: int64



**Bien, repasando de nuevo, un mensaje se considera que proviene de un suscriptor en estos 
casos:**

**1- su origen plasmado en la variable "Enviado/por", resuelto anteriormente**

**2- contenga frases asociadas con el servicio de internet, en especial las siguientes
son trascendentes: "no hay conexion", "no hay internet", "cuanto es la mensualidad"**

**3- tomar en cuenta comodin por error ortografico comun "interne" y otros


In [11]:
# palabras de interes dado el contexto en que se esta trabajando:
# 1 interne (comodin de internet):conexion

mensajes["internet"] = np.ones(mensajes.shape[0], dtype=int)
mensajes.loc[mensajes['Texto'].str.contains('interne'), "internet"] = 0 
print(mensajes.internet.value_counts())
print(pd.crosstab(mensajes["internet"], columns=mensajes["es_suscriptor"]))

mensajes["conexion"] = np.ones(mensajes.shape[0], dtype=int)
mensajes.loc[mensajes['Texto'].str.contains('conexion'), "conexion"] = 0 
print(mensajes.internet.value_counts())
print(pd.crosstab(mensajes["conexion"], columns=mensajes["es_suscriptor"]))


# 1 mes o mensualidad (comodin de "cuanto"):
mensajes["cuanto"] = np.ones(mensajes.shape[0], dtype=int)
mensajes.loc[mensajes['Texto'].str.contains('cuanto'), "cuanto"] = 0 
print(mensajes.internet.value_counts())
print(pd.crosstab(mensajes["cuanto"], columns=mensajes["es_suscriptor"]))




1    728
0    320
Name: internet, dtype: int64
es_suscriptor    0    1
internet               
0              134  186
1              125  603
1    728
0    320
Name: internet, dtype: int64
es_suscriptor    0    1
conexion               
0              24   18 
1              235  771
1    728
0    320
Name: internet, dtype: int64
es_suscriptor    0    1
cuanto                 
0              17   49 
1              242  740


In [13]:
# listando un subgrupo de interes a partir de la tabulacion cruzada anterior

import operator

mensajes.loc[operator.and_(mensajes.es_suscriptor == 1, mensajes.internet == 0)]


,Enviado/por,Fecha/Hora,Texto,ts,tem,es_suscriptor,tipo_mensaje,internet,conexion,cuanto
8,04147333928,2018-08-31 19:31:41,hola luis como estas? yo abusando de tu confianza puedes colocarme el internet sin falta paso manana te cancelo soy marithza estaba esperando una transferencia ya se me hizo efectiva,70301,29143.0,1,0,0,1,1
15,04248636635,2018-08-30 17:32:24,buenas tarde sr luis sera q me puede activar el.internet y me.dice cuando es lo del mes para trsnsferircelos es genesis la de.cosovo pero.xfa activamelo para aserte la tranaferencia es no tengo.telefono pqra aserla,63144,28079.0,1,0,0,1,1
19,04248636635,2018-08-31 07:31:37,buenos dias sr luis si activo el internet,27097,49282.0,1,0,0,1,1
20,04248636635,2018-08-31 08:32:31,buen dia sr luis xfavor activeme el internet y de aca de una le transfiero ay sr luis lo necesito o tienes punto aya abajo,30751,3654.0,1,0,0,1,1
40,04247774757,2018-03-09 09:18:16,buenos dias senor luis es betty del bloque 8 queria preguntarle sime puede chequear el internet del apto que no lo se conectar,33496,14579.0,1,0,0,1,1
69,04247774757,2018-03-09 09:18:16,buenos dias senor luis es betty del bloque 8 queria preguntarle sime puede chequear el internet del apto que no lo se conectar,33496,14579.0,1,0,0,1,1
71,04248636635,2018-01-09 17:09:02,sr luis ud me volvio a kitar el internet ayer la paguina del banco venezuela no me keria abrir para transferorle xeso no se la ise iva ir orota aserla yme kitaste de.nuevo el intern,61742,28243.0,1,0,0,1,1
88,04164947520,2018-04-09 20:53:05,buenas noche sr luis como esta soy la sra zublay es para decirle q aqui en el apartamento no hay internet es para saber si hay o no hay.,75185,2498.0,1,0,0,1,1
93,04247345175,2018-03-09 18:53:08,hola senor luis cm estas cuanto es lo dl internet es la senora mery,67988,8755.0,1,0,0,1,0
94,04247345175,2018-03-09 18:53:29,hola senor luis cm estas cuanto es lo dl internet es la senora mery,68009,21.0,1,0,0,1,0
